In [219]:
import pandas as pd
import seaborn as sn
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

In [220]:
data=pd.read_csv('final.csv',low_memory=False)

In [221]:
data

,FloatID,mbariID,region,Station,Type,time,longitude_degrees_east,latitude_degrees_north,latitude_QF,Pressure_dbar,pressure_QF,Temperature_Celcius,temperature_QF,salinity_pss,salinity_QF,Sigma_theta_kg_m_pow_minus_3,sigma_theta_QF,depth_m,depth_QF,oxygen_mu_molkg_pow_minus_1,oxygen_QF,oxygen_saturation_percentage,oxygen saturation_QF,nitrate_mu_mol_kg_pow_minus_1,nitrate_QF,chl_a_mg_m_pow_minus_3,chl_a_QF,b_bp700_m_pow_minus_1,b_bp700_QF,POC_milli_mol_m_pow_minus_3,POC_QF,pH_insitu_total,pH_insitu_QF,pH_25C_total,pH_25C_QF,TALK_micro_mol_kg_pow_minus_1,TALK_QF,DIC_micro_mol_kg_pow_minus_1,DIC_QF,pCO2_micro_atm,pCO2_QF,b_bp532_m_pow_minus_1,b_bp532_QF,CDOM_ppb,CDOM_QF
0,1901378.0,1901378QC,N. Atlantic,1.0,C,2009-10-12T06:39:00Z,295.799,31.689,0.0,401.53,0.0,18.1378,0.0,36.5433,0.0,26.439,0.0,398.356,0.0,2.052700e+02,0.0,8.890000e+01,0.0,5.980000e+00,0.0,-1.000000e+10,1.0,-1.000000e+10,1.0,-1.000000e+10,1.0,-1.000000e+10,1.0,-1.000000e+10,1.0,-1.000000e+10,1.0,-1.000000e+10,1.0,-1.000000e+10,1.0,-1.000000e+10,1.0,-1.000000e+10,1.0
1,1901378.0,1901378QC,N. Atlantic,1.0,C,2009-10-12T06:39:00Z,295.799,31.689,0.0,381.67,0.0,18.2043,0.0,36.5514,0.0,26.428,0.0,378.671,0.0,2.051000e+02,0.0,8.890000e+01,0.0,5.480000e+00,0.0,-1.000000e+10,1.0,-1.000000e+10,1.0,-1.000000e+10,1.0,-1.000000e+10,1.0,-1.000000e+10,1.0,-1.000000e+10,1.0,-1.000000e+10,1.0,-1.000000e+10,1.0,-1.000000e+10,1.0,-1.000000e+10,1.0
2,1901378.0,1901378QC,N. Atlantic,1.0,C,2009-10-12T06:39:00Z,295.799,31.689,0.0,360.87,0.0,18.2639,0.0,36.5546,0.0,26.414,0.0,358.052,0.0,2.067300e+02,0.0,8.970000e+01,0.0,5.390000e+00,0.0,-1.000000e+10,1.0,-1.000000e+10,1.0,-1.000000e+10,1.0,-1.000000e+10,1.0,-1.000000e+10,1.0,-1.000000e+10,1.0,-1.000000e+10,1.0,-1.000000e+10,1.0,-1.000000e+10,1.0,-1.000000e+10,1.0
3,1901378.0,1901378QC,N. Atlantic,1.0,C,2009-10-12T06:39:00Z,295.799,31.689,0.0,350.63,0.0,18.3001,0.0,36.5556,0.0,26.406,0.0,347.901,0.0,2.065400e+02,0.0,8.970000e+01,0.0,5.560000e+00,0.0,-1.000000e+10,1.0,-1.000000e+10,1.0,-1.000000e+10,1.0,-1.000000e+10,1.0,-1.000000e+10,1.0,-1.000000e+10,1.0,-1.000000e+10,1.0,-1.000000e+10,1.0,-1.000000e+10,1.0,-1.000000e+10,1.0
4,1901378.0,1901378QC,N. Atlantic,1.0,C,2009-10-12T06:39:00Z,295.799,31.689,0.0,341.21,0.0,18.3424,0.0,36.5606,0.0,26.398,0.0,338.562,0.0,2.061600e+02,0.0,8.960000e+01,0.0,5.150000e+00,0.0,-1.000000e+10,1.0,-1.000000e+10,1.0,-1.000000e+10,1.0,-1.000000e+10,1.0,-1.000000e+10,1.0,-1.000000e+10,1.0,-1.000000e+10,1.0,-1.000000e+10,1.0,-1.000000e+10,1.0,-1.000000e+10,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
301716,4903026.0,4903026QC,N. Pacific,92.0,C,2022-05-19T10:07:00Z,237.741,34.997,0.0,1.92,0.0,12.1478,0.0,33.5278,0.0,25.418,0.0,1.906,0.0,2.643700e+02,0.0,9.970000e+01,0.0,-1.000000e+10,1.0,4.884000e-01,4.0,1.741000e-03,0.0,4.780000e+00,0.0,2.520660e+01,8.0,-1.000000e+10,1.0,2.240000e+03,4.0,-1.000000e+10,1.0,-1.000000e+10,1.0,-1.000000e+10,1.0,-1.000000e+10,1.0
301717,4903026.0,4903026QC,N. Pacific,92.0,C,2022-05-19T10:07:00Z,237.741,34.997,0.0,1.00,0.0,12.1420,0.0,33.5290,0.0,25.420,0.0,0.993,0.0,-1.000000e+10,1.0,-1.000000e+10,1.0,-1.000000e+10,1.0,-1.000000e+10,1.0,-1.000000e+10,1.0,-1.000000e+10,1.0,-1.000000e+10,1.0,-1.000000e+10,1.0,2.239000e+03,4.0,-1.000000e+10,1.0,-1.000000e+10,1.0,-1.000000e+10,1.0,-1.000000e+10,1.0
301718,4903026.0,4903026QC,N. Pacific,92.0,C,2022-05-19T10:07:00Z,237.741,34.997,0.0,0.92,0.0,12.1410,0.0,33.5304,0.0,25.422,0.0,0.913,0.0,-1.000000e+10,1.0,-1.000000e+10,1.0,-1.000000e+10,1.0,-1.000000e+10,1.0,-1.000000e+10,1.0,-1.000000e+10,1.0,-1.000000e+10,1.0,-1.000000e+10,1.0,2.239000e+03,4.0,-1.000000e+10,1.0,-1.000000e+10,1.0,-1.000000e+10,1.0,-1.000000e+10,1.0
301719,4903026.0,4903026QC,N. Pacific,92.0,C,2022-05-19T10:07:00Z,237.741,34.997,0.0,0.80,0.0,12.1430,0.0,33.5294,0.0,25.421,0.0,0.794,0.0,-1.000000e+10,1.0,-1.000000e+10,1.0,-1.000000e+

In [222]:
data.dtypes

FloatID                          float64
mbariID                           object
region                            object
Station                          float64
Type                              object
time                              object
longitude_degrees_east           float64
latitude_degrees_north           float64
latitude_QF                      float64
Pressure_dbar                    float64
pressure_QF                      float64
Temperature_Celcius              float64
temperature_QF                   float64
salinity_pss                     float64
salinity_QF                      float64
Sigma_theta_kg_m_pow_minus_3     float64
sigma_theta_QF                   float64
depth_m                          float64
depth_QF                         float64
oxygen_mu_molkg_pow_minus_1      float64
oxygen_QF                        float64
oxygen_saturation_percentage     float64
oxygen saturation_QF             float64
nitrate_mu_mol_kg_pow_minus_1    float64
nitrate_QF      

In [223]:
column_type=['Pressure_dbar','Temperature_Celcius','salinity_pss','Sigma_theta_kg_m_pow_minus_3','depth_m','oxygen_mu_molkg_pow_minus_1',
'oxygen_saturation_percentage','nitrate_mu_mol_kg_pow_minus_1',
'chl_a_mg_m_pow_minus_3','b_bp700_m_pow_minus_1','POC_milli_mol_m_pow_minus_3',
'pH_insitu_total','pH_25C_total','TALK_micro_mol_kg_pow_minus_1',
'DIC_micro_mol_kg_pow_minus_1','pCO2_micro_atm','b_bp532_m_pow_minus_1',
'CDOM_ppb']

In [224]:
# action for conversion
# for col in column_type:
#     data[col]=data[col].astype(float)

In [225]:
data.dtypes

FloatID                          float64
mbariID                           object
region                            object
Station                          float64
Type                              object
time                              object
longitude_degrees_east           float64
latitude_degrees_north           float64
latitude_QF                      float64
Pressure_dbar                    float64
pressure_QF                      float64
Temperature_Celcius              float64
temperature_QF                   float64
salinity_pss                     float64
salinity_QF                      float64
Sigma_theta_kg_m_pow_minus_3     float64
sigma_theta_QF                   float64
depth_m                          float64
depth_QF                         float64
oxygen_mu_molkg_pow_minus_1      float64
oxygen_QF                        float64
oxygen_saturation_percentage     float64
oxygen saturation_QF             float64
nitrate_mu_mol_kg_pow_minus_1    float64
nitrate_QF      

In [226]:
# finding null values in each columns
data.isna().sum()

FloatID                          0
mbariID                          0
region                           0
Station                          0
Type                             0
time                             0
longitude_degrees_east           0
latitude_degrees_north           0
latitude_QF                      0
Pressure_dbar                    0
pressure_QF                      0
Temperature_Celcius              0
temperature_QF                   0
salinity_pss                     0
salinity_QF                      0
Sigma_theta_kg_m_pow_minus_3     0
sigma_theta_QF                   0
depth_m                          0
depth_QF                         0
oxygen_mu_molkg_pow_minus_1      0
oxygen_QF                        0
oxygen_saturation_percentage     0
oxygen saturation_QF             0
nitrate_mu_mol_kg_pow_minus_1    0
nitrate_QF                       0
chl_a_mg_m_pow_minus_3           0
chl_a_QF                         0
b_bp700_m_pow_minus_1            0
b_bp700_QF          

In [227]:
# droppoing mbariID column as it contains only 1 value
data.drop(['mbariID'],axis=1,inplace=True)

In [228]:
data.region.unique()

array(['N. Atlantic', 'N. Pacific'], dtype=object)

In [229]:
# region column has been done label encoding

# might change to another feature encoding method if required
label_encoder = preprocessing.LabelEncoder()
data['region']= label_encoder.fit_transform(data['region'])
  

In [230]:
data.Type.unique()

array(['C'], dtype=object)

In [231]:
data.drop(['Type','pressure_QF'],axis=1,inplace=True)

In [232]:
## time need to be converted


data.time.unique()

array(['2009-10-12T06:39:00Z', '2009-10-13T06:05:00Z',
       '2009-10-14T06:36:00Z', '2009-10-15T07:15:00Z',
       '2009-10-16T07:33:00Z', '2009-10-17T09:15:00Z',
       '2009-10-22T12:49:00Z', '2009-10-27T16:16:00Z',
       '2009-11-01T18:37:00Z', '2009-11-06T22:08:00Z',
       '2009-11-12T02:13:00Z', '2009-11-17T06:07:00Z',
       '2009-11-22T09:45:00Z', '2009-11-27T13:34:00Z',
       '2009-12-02T17:04:00Z', '2009-12-07T21:40:00Z',
       '2009-12-13T01:13:00Z', '2009-12-18T05:23:00Z',
       '2009-12-23T10:00:00Z', '2009-12-28T14:13:00Z',
       '2010-01-01T19:16:00Z', '2010-01-07T23:49:00Z',
       '2010-01-13T03:39:00Z', '2010-01-18T07:10:00Z',
       '2010-01-23T10:44:00Z', '2010-01-28T14:16:00Z',
       '2010-02-02T17:21:00Z', '2010-02-07T20:17:00Z',
       '2010-02-12T23:33:00Z', '2010-02-18T03:29:00Z',
       '2010-02-23T07:06:00Z', '2010-02-28T11:13:00Z',
       '2010-03-05T15:42:00Z', '2010-03-10T20:03:00Z',
       '2010-03-16T01:01:00Z', '2010-03-21T04:26:00Z',
       '20

In [233]:
test,train = train_test_split(data,
                          random_state=14,
                          test_size=0.7)

In [234]:
# a=pd.read_csv('1902381QC.csv',low_memory=False)
# a.nitrate.unique().min()



In [235]:
for col in data.columns:
    print("min value for ",col," column: ",data[col].unique().min())
    print("max value for ",col," column: ",data[col].unique().max())

min value for  FloatID  column:  1901378.0
max value for  FloatID  column:  4903365.0
min value for  region  column:  0
max value for  region  column:  1
min value for  Station  column:  1.0
max value for  Station  column:  293.0
min value for  time  column:  2009-10-12T06:39:00Z
max value for  time  column:  2022-05-19T10:07:00Z
min value for  longitude_degrees_east  column:  185.131
max value for  longitude_degrees_east  column:  345.74
min value for  latitude_degrees_north  column:  15.186
max value for  latitude_degrees_north  column:  61.477
min value for  latitude_QF  column:  0.0
max value for  latitude_QF  column:  4.0
min value for  Pressure_dbar  column:  0.2
max value for  Pressure_dbar  column:  2003.68
min value for  Temperature_Celcius  column:  2.055
max value for  Temperature_Celcius  column:  29.0677
min value for  temperature_QF  column:  0.0
max value for  temperature_QF  column:  0.0
min value for  salinity_pss  column:  32.807
max value for  salinity_pss  column:  